#### Names of people in the group

Please write the names of the people in your group in the next cell.

Joachim Maksim

Jørgen Nordli Katralen

In [0]:
# Loading modules that we need
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

#### The problem: mining the interests of experts

The primary role of a questions and answering platform such as Stack Exchange is to connect two types of people. Namely, people who have questions in areas such as computer science or data science and knowledgeable people who can answer those questions reliably. Let's call the first category of people' knowledge seekers' and the second one 'expert users' or 'experts' for short.

Here we want to answer a question related to the diversity of topics that experts are interested in using our data. We want to know if expert users only answer questions in a specific set of topics or their interests include a wide variety of topics.

To answer the above question, we will compute the correlation between a user's expertise level and the diversity of topics of questions they have answered. The first step is to define two variables (or measures); first for 'user expertise level' and then for 'user interest diversity'. Then we will use the Pearson correlation coefficient to measure the linear correlation between the two variables. We define the variables as:

   - VariableA (the measure of user expertise level). We will use the 'Reputation' column from 'users' table, which according to Stack Exchange's documentation "is a rough measurement of how much the community trusts you; it is earned by convincing your peers that you know what you're talking about" as an indicator of a user's expertise level on the platform. 

   - VariableB (The measure of user interest diversity). We measure the diversity of a user's interests by computing the total number of distinct tags associated with the questions each user has answered divided by the total number of unique tags which is 638.

Compute the Pearson correlation coefficient between VariableA and VariableB, and based on the result you've got, answer the following question: 

     Do expert users have specif interests or do they have general interests?

Please explain your reasoning on how you reached your answer.

You should use Apache Spark API for your implementation. You can use the Spark implementation of the Pearson correlation coefficient.

In [0]:
# Variable A: We order users on reputation
a_df = users_df.select("Id", "DisplayName", "Reputation").orderBy("Reputation", ascending=False)
a_df.show()

+-----+--------------------+----------+
|   Id|         DisplayName|Reputation|
+-----+--------------------+----------+
|  836|         Neil Slater|     24229|
| 8820|        Martin Thoma|     15185|
|28175|               Media|     11793|
|45264|             n1k31t4|     11711|
|64377|               Erwan|     10346|
| 1330|      Brian Spiering|     10037|
|  381|                Emre|      9821|
|14904|    Jan van der Vegt|      8410|
|14675|              ncasas|      8206|
|29587|            JahKnows|      7613|
|11097|             Dawny33|      7496|
|  924|Has QUIT--Anony-M...|      7248|
|67328|           Esmailian|      7044|
| 2452|     Aleksandr Blekh|      6388|
|55122|        Ben Reiniger|      6349|
|21560|             Hendrik|      6337|
| 9420|               AN6U5|      6168|
|34269|            Djib2011|      6025|
|   21|           Sean Owen|      5904|
|23305|                 oW_|      5379|
+-----+--------------------+----------+
only showing top 20 rows



In [0]:
# Variable B
# We prepare the posts dataframe for a future join operation
posts_df_2 = posts_df.select("Tags", "Id")

# We want to generate a table with columns (OwnerUserId, AnsweredPostId, Tags) where we have the owner user, the user's answer, and the tags of the question being answered.

# 1. We filter on PostType = Answer and extract the OwnerUser and the ParentId (the question being answered).
answeredPosts = posts_df.filter(posts_df.PostTypeId == 2).select("OwnerUserId", "ParentId")

# 2. We want to include tags, so we do an inner join with posts_df_2, matching Id with ParentId.
answeredPosts = answeredPosts.join(posts_df_2, posts_df.ParentId == posts_df_2.Id, 'inner')

# 3. We select and rename fields.
answeredPosts = answeredPosts.select("OwnerUserId", "Id", "Tags").withColumnRenamed("Id", "AnsweredPostId")

# Now, the tags column contains a string of tags: "<tag1><tag2>". 

# 4. We convert the tags string into a list: ["tag1", "tag2"]
b_df = answeredPosts.rdd.map(lambda x: Row(OwnerUserId=x[0], AnsweredPostId=x[1], Tags=x[2].lstrip("<").rstrip(">").split("><"))).toDF()

# 5. We create a new row for each tag in the list, maintaining the same OwnerUserId for each generated / "exploded" row. We need this to remove duplicate entries in the next step.
b_df = b_df.select("OwnerUserId", explode("Tags"))

# 6. We remove duplicate entries and count the number of rows for each user. We then calculate the interest diversity by dividing the count by the number of existing tags.
b_df = b_df.distinct().rdd.groupByKey().map(lambda x: Row(User=x[0], InterestDiversity=len(x[1]) / 638)).toDF()

b_df.show()

+-----+--------------------+
| User|   InterestDiversity|
+-----+--------------------+
| 1015|0.007836990595611285|
| 9768| 0.01567398119122257|
|  241|  0.0109717868338558|
|10352|  0.0219435736677116|
|   59| 0.02664576802507837|
| 3100|0.006269592476489028|
| 1011|0.025078369905956112|
| 1176|0.001567398119122257|
| 9123|   0.109717868338558|
| 8878| 0.21786833855799373|
|  847| 0.05642633228840126|
| 8085|  0.0219435736677116|
| 7969|0.023510971786833857|
|  525|0.029780564263322883|
|12328|0.007836990595611285|
|  836|  0.2476489028213166|
| 9420| 0.11128526645768025|
|10148|0.006269592476489028|
|37567|0.004702194357366771|
|10517| 0.03134796238244514|
+-----+--------------------+
only showing top 20 rows



In [0]:
# Do expert users only answer questions in a specific set of topics, or do their interests include a wide variety of topics?

# Since the question states that we are allowed to use the Spark implementation of the pearson coefficient, we will use the "corr" method:
joined_table = a_df.join(b_df, a_df.Id == b_df.User, "inner").select("User", "DisplayName", "Reputation", "InterestDiversity")
pearson = joined_table.corr("Reputation", "InterestDiversity")
print(f"Pearson('Reputation', 'InterestDiversity') = {pearson}\n")

# We print the top 20 users (ordered by reputation) and 20 users with low reputation, to check if we can clearly spot a correlation between reputation and interest diversity
print("Top 20 users by reputation:")
joined_table.orderBy("Reputation", ascending=False).show()

print("Bottom 20 users with reputation above 98:")
joined_table.filter(joined_table.Reputation > 98).orderBy("Reputation", ascending=True).show()


Pearson('Reputation', 'InterestDiversity') = 0.7217677648623005

Top 20 users by reputation:
+-----+--------------------+----------+--------------------+
| User|         DisplayName|Reputation|   InterestDiversity|
+-----+--------------------+----------+--------------------+
|  836|         Neil Slater|     24229|  0.2476489028213166|
| 8820|        Martin Thoma|     15185|  0.1426332288401254|
|28175|               Media|     11793| 0.23981191222570533|
|45264|             n1k31t4|     11711| 0.32445141065830724|
|64377|               Erwan|     10346| 0.46551724137931033|
| 1330|      Brian Spiering|     10037| 0.48746081504702193|
|  381|                Emre|      9821| 0.21003134796238246|
|14904|    Jan van der Vegt|      8410|  0.1974921630094044|
|14675|              ncasas|      8206| 0.31347962382445144|
|29587|            JahKnows|      7613|  0.2115987460815047|
|11097|             Dawny33|      7496| 0.15203761755485892|
|  924|Has QUIT--Anony-M...|      7248|  0.2115987460

### Conclusion
We see that there is a rather strong correlation between a user's level of expertise (measured with reputation) and the user's interest diversity. The pearson correlation between the Reputation and InterestDiversity columns was 0.72176... which is quite high. We can also clearly see the correlation when comparing the table of 20 users with high reputation to the table of 20 users with low reputation. Users with low reputation tend to have a much lower interest diversity than users with high reputation.

However, the strong correlation may be biased, as the reputation score may not necessarily be representative of the expertise level of the users. Users with many answers will naturally have a higher reputation. The ratio of likes and dislikes on user posts may also indicate the quality of the responses that the users give, and thus reflect their level of expertise. Ideally, we would normalize the reputation score and take into account the aforementioned factors.

Further analysis could include assigning weights to badges, and adjusting the users' reputation based on their assigned badge. For example, Students would naturally have a lower level of expertise than Teachers.